In [1]:
import pyodbc
import mysql.connector
import dask.dataframe as dd
import pandas as pd
# from scripts.db_utils import create_ebay_connection
import os 
from dotenv import load_dotenv

load_dotenv()

def create_ebay_connection():
    """Create connection to the hosted SQL Server database for IP addresses."""
    try:
        connection = pyodbc.connect(
            f"DRIVER={{MySQL ODBC 9.1 Unicode Driver}};"
            f"SERVER={os.getenv('EBAY_DB_HOST')};"
            f"DATABASE={os.getenv('EBAY_DB_NAME')};"
            f"UID={os.getenv('EBAY_DB_USER')};"
            f"PWD={os.getenv('EBAY_DB_PASSWORD')}"
        )
        return connection
    except pyodbc.Error as e:
        print(f"Error connecting to hosted database: {e}")
        return None
# Database connection parameters
conn = create_ebay_connection()

query = "SELECT * FROM tbl_iptrace_user_activity"




def fetch_data_in_chunks(conn, query, chunk_size=10000):
    """
    Fetch data from the database in chunks, then convert to Dask DataFrame.
    """
    offset = 0
    while True:
        # Modify the query to include pagination using LIMIT and OFFSET for chunks
        paginated_query = f"{query} LIMIT {chunk_size} OFFSET {offset}"
        
        # Fetch data in chunks using pandas (it handles column and row alignment internally)
        df_chunk = pd.read_sql_query(paginated_query, conn)
        
        if df_chunk.empty:
            break
        
        # Convert to Dask DataFrame
        dask_df = dd.from_pandas(df_chunk, npartitions=4)
        
        yield dask_df
        
        offset += chunk_size


for dask_df_chunk in fetch_data_in_chunks(conn, query, chunk_size=10000):
    # Print the head of the current Dask DataFrame chunk
    print(dask_df_chunk.head())


# print(conn)

Columns: ['id', 'CLNT_RMT_IP', 'ROI_CLICK_EVENT_TS', 'BRWSR_NAME', 'ams_trans_rsn_cd', 'ams_pblshr_id', 'buyer_id', 'is_vpn', 'click_id']
First row: (1, '77.111.247.168', datetime.datetime(2020, 8, 2, 18, 37, 1), 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 OPR/69.0.3686.95', 0, '5574672411', 'valen_949', 1, '209245484389035777')


ValueError: Shape of passed values is (10000, 1), indices imply (10000, 9)